# 使用技术指标来识别市场结构

市场结构：将价格行为划分为趋势和震荡两种类型。

能够识别市场结构的关键技术指标：

- Market Meanness Index (MMI)
- Fractal Dimension

**Market Meanness Index (MMI)**

取值范围是0-100，当指标处于下降趋势，认为市场处于趋势状态；当指标处于上升趋势，认为市场处于震荡状态。

**Fractal Dimension**

分形维数：取值范围是0-2，当指标高于阈值，认为市场处于震荡行情，当指标低于阈值，认为市场处于趋势行情。阈值的选择根资产相关，常见阈值为1.2-1.5。

In [21]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


def calculate_mmi(data: pd.Series, period: int) -> pd.Series:
    """
    Calculate the Market Meanness Index (MMI) for asset prices.

    Args:
        data (pd.Series): The price series.
        period (int): The lookback period of the indicator, must be greater than 1.

    Returns:
        pd.Series: The Market Meanness Index (MMI) values.
    """
    # 参数验证
    if not isinstance(data, pd.Series):
        raise TypeError("data must be a pandas Series")
    if not isinstance(period, int):
        raise TypeError("period must be an integer")
    if period < 2:
        raise ValueError("period must be greater than 1")
    if len(data) < period:
        raise ValueError("data length must be >= period")

    def _mmi(data: np.ndarray) -> float:
        # 反转数据，索引0表示最新数据
        series = data[::-1]

        # 计算中位数
        median = np.median(series)

        nh = nl = 0
        for i in range(1, len(series)):
            if series[i] > median and series[i] > series[i - 1]:
                nl += 1
            elif series[i] < median and series[i] < series[i - 1]:
                nh += 1

        # 计算MMI
        return 100.0 * (nl + nh) / (len(series) - 1)

    return data.rolling(window=period, min_periods=period).apply(_mmi, raw=True)


def calculate_fractal_dimension(data: pd.Series, period: int) -> pd.Series:
    """
    Calculate the Fractal Dimension for asset prices.

    Args:
        data (pd.Series): The price series.
        period (int): The lookback period of the indicator, must be greater than 1
            and will be converted to even number if odd.

    Returns:
        pd.Series: The Fractal Dimension values.
    """
    if not isinstance(data, pd.Series):
        raise TypeError("data must be a pandas Series")
    if not isinstance(period, int):
        raise TypeError("period must be an integer")
    if period < 2:
        raise ValueError("period must be greater than 1")
    if len(data) < period:
        raise ValueError("data length must be >= period")

    def _fractal_dimension(data: np.ndarray) -> float:
        # 反转数据，索引0表示最新数据
        series = data[::-1]

        # 计算半周期
        period = len(data)
        period2 = period // 2

        n1 = (max(series[0:period2]) - min(series[0:period2])) / period2
        n2 = (max(series[period2:period]) - min(series[period2:period])) / period2
        n3 = (max(series[0:period]) - min(series[0:period])) / period

        if n1 + n2 <= 0 or n3 <= 0:
            return 1.0

        return (np.log(n1 + n2) - np.log(n3)) / np.log(2)

    # 确保period是偶数
    # 如果period是奇数，将其减1，如果period是偶数，保持不变
    period = period & ~1

    return data.rolling(window=period, min_periods=period).apply(
        _fractal_dimension, raw=True
    )

读取数据

In [17]:
file_path = "../data/yahoo/Bitcoin.csv"
df = pd.read_csv(file_path, parse_dates=True, index_col=0)
df.round(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.86,468.17,452.42,457.33,457.33,21056800
2014-09-18,456.86,456.86,413.10,424.44,424.44,34483200
2014-09-19,424.10,427.83,384.53,394.80,394.80,37919700
2014-09-20,394.67,423.30,389.88,408.90,408.90,36863600
2014-09-21,408.08,412.43,393.18,398.82,398.82,26580100
...,...,...,...,...,...,...
2024-12-13,100046.65,101888.80,99233.28,101459.26,101459.26,56894751583
2024-12-14,101451.44,102618.88,100634.05,101372.97,101372.97,40422968793
2024-12-15,101373.53,105047.54,101227.03,104298.70,104298.70,51145914137


计算指标

In [39]:
mmi_period = 100
fd_period = 100

df["mmi"] = calculate_mmi(df["Close"], mmi_period)
df["fd"] = calculate_fractal_dimension(df["Close"], fd_period)

可视化指标

In [40]:
# 创建子图
fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=(
        "Bitcoin Price",
        "Market Meanness Index",
        "Fractal Dimension",
    ),
)

# 添加收盘价
fig.add_trace(
    go.Scatter(x=df.index, y=df["Close"]),
    row=1,
    col=1,
)

# 添加MMI
fig.add_trace(
    go.Scatter(x=df.index, y=df["mmi"], marker_color="orange"),
    row=2,
    col=1,
)

# 添加分形维数
fig.add_trace(
    go.Scatter(x=df.index, y=df["fd"], marker_color="purple"),
    row=3,
    col=1,
)
fig.add_hline(y=1.5, line_dash="dot", line_color="grey", row=3, col=1)

# 更新布局
fig.update_layout(width=1200, height=800, showlegend=False)

fig.show()